### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# Create a dataframe with a dictionary to store calculations
district_summary = [{"Total Schools":0, "Total Students":0, "Total Budget":0.0, "Average Math Score":0.0, "Average Reading Score":0.0, "%Passing Math":0.0, "%Passing Reading":0.0, "%Overall Passing Rate":0.0}]
district_summary_df = pd.DataFrame(district_summary)

# Calculate the total number of schools
district_summary_df["Total Schools"] = len(school_data_complete["school_name"].unique())

# Calculate the total number of students
district_summary_df["Total Students"] = len(school_data_complete["Student ID"].unique())

# Calculate the total budget
#district_summary_df["Total Budget"] = sum(school_data_complete.loc[school_data_complete["School ID"].unique(),"budget"])
district_summary_df["Total Budget"] = sum(school_data["budget"])

# Calculate the average math score
district_summary_df["Average Math Score"] = school_data_complete["math_score"].mean()

# Calculate the average reading score
district_summary_df["Average Reading Score"] = school_data_complete["reading_score"].mean()

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
district_summary_df["%Overall Passing Rate"] = (school_data_complete["math_score"].mean() + school_data_complete["reading_score"].mean()) / 2

# Calculate the percentage of students with a passing math score (70 or greater)
passing_students = school_data_complete["math_score"].loc[school_data_complete["math_score"] >= 70].count()
district_summary_df["%Passing Math"] = 100 * passing_students / district_summary_df["Total Students"]

# Calculate the percentage of students with a passing reading score (70 or greater)
passing_students = school_data_complete["reading_score"].loc[school_data_complete["reading_score"] >= 70].count()
district_summary_df["%Passing Reading"] = 100 * passing_students / district_summary_df["Total Students"]

# Clean up the output

district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["%Passing Math"] = district_summary_df["%Passing Math"].map("{:.1f}%".format)
district_summary_df["%Passing Reading"] = district_summary_df["%Passing Reading"].map("{:.1f}%".format)
district_summary_df["%Overall Passing Rate"] = district_summary_df["%Overall Passing Rate"].map("{:.1f}%".format)


In [4]:
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing Rate
0,15,"39,170","$24,649,428.00",79.0,81.9,75.0%,85.8%,80.4%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [64]:
# Sort school and student Data Files by school_name
school_data = school_data.sort_values("school_name")
student_data = student_data.sort_values("school_name")

school_summary = pd.DataFrame ({
    "School Name": [],
    "School Type": [],
    "Total Students": [],
    "Total School Budget": [],
    "Per Student Budget": [],
    "Average Math Score": [],
    "Average Reading Score": [],
    "% Passing Math": [],
    "% Passing Reading": [],
    "% Overall Passing Rate": []
})

school_summary["School Name"] = school_data["school_name"]

school_summary["School Type"] = school_data["type"]

school_summary["Total Students"] = school_data["size"]

school_summary["Total School Budget"] = school_data["budget"]

school_summary["Per Student Budget"] = [i / j for i, j in zip(school_data["budget"], school_data["size"])]

avg_math_score = school_data_complete.groupby("school_name")["math_score"].sum() / \
    school_data_complete.groupby("school_name")["Student ID"].count()
school_summary["Average Math Score"] = [x for x in avg_math_score]

avg_reading_score = school_data_complete.groupby("school_name")["reading_score"].sum() / \
    school_data_complete.groupby("school_name")["Student ID"].count()
school_summary["Average Reading Score"] = [x for x in avg_reading_score]

passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
passing_math.groupby("school_name")["math_score"].count()
pct_passing_math = 100 * passing_math.groupby("school_name")["math_score"].count() / \
    school_data_complete.groupby("school_name")["Student ID"].count()
school_summary["% Passing Math"] = [x for x in pct_passing_math]

passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
passing_reading.groupby("school_name")["reading_score"].count()
pct_passing_reading = 100 * passing_reading.groupby("school_name")["reading_score"].count() / \
    school_data_complete.groupby("school_name")["Student ID"].count()
school_summary["% Passing Reading"] = [x for x in pct_passing_reading]

passing_overall = (school_summary["% Passing Math"] + school_summary["% Passing Reading"]) / 2
school_summary["% Overall Passing Rate"] = [x for x in passing_overall]

# Clean up the output

school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)

# Sort and display results to screen
school_summary_sort = school_summary.sort_values("% Overall Passing Rate", ascending=False)
school_summary_sort.head(5)


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [65]:
# Sort and display results to screen
school_summary_sort = school_summary.sort_values("% Overall Passing Rate")
school_summary_sort.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
13,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [90]:
# Create a pandas series for each grade. Hint: use a conditional statement.
ninth = school_data_complete.loc[school_data_complete["grade"] == "9th"]
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th"]
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th"]
twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th"]

# Group each series by school
ninth_math = ninth.groupby("school_name")["math_score"].sum() / \
    ninth.groupby("school_name")["Student ID"].count()
tenth_math = tenth.groupby("school_name")["math_score"].sum() / \
    tenth.groupby("school_name")["Student ID"].count()
eleventh_math = eleventh.groupby("school_name")["math_score"].sum() / \
    eleventh.groupby("school_name")["Student ID"].count()
twelfth_math = twelfth.groupby("school_name")["math_score"].sum() / \
    twelfth.groupby("school_name")["Student ID"].count()

# Combine the series into a dataframe
math_summary = pd.DataFrame ({ "School Name": "", "9th": [], "10th": [],"11th": [], "12th": []
})
math_summary["School Name"] = school_data["school_name"]
math_summary["9th"] = [x for x in ninth_math]
math_summary["10th"] = [x for x in tenth_math]
math_summary["11th"] = [x for x in eleventh_math]
math_summary["12th"] = [x for x in twelfth_math]
math_summary = math_summary.set_index("School Name")
math_summary.index.name = None

# Optional: give the displayed data cleaner formatting

math_summary["9th"] = math_summary["9th"].map("{:.1f}%".format)
math_summary["10th"] = math_summary["10th"].map("{:.1f}%".format)
math_summary["11th"] = math_summary["11th"].map("{:.1f}%".format)
math_summary["12th"] = math_summary["12th"].map("{:.1f}%".format)


math_summary

,9th,10th,11th,12th
Bailey High School,77.1%,77.0%,77.5%,76.5%
Cabrera High School,83.1%,83.2%,82.8%,83.3%
Figueroa High School,76.4%,76.5%,76.9%,77.2%
Ford High School,77.4%,77.7%,76.9%,76.2%
Griffin High School,82.0%,84.2%,83.8%,83.4%
Hernandez High School,77.4%,77.3%,77.1%,77.2%
Holden High School,83.8%,83.4%,85.0%,82.9%
Huang High School,77.0%,75.9%,76.4%,77.2%
Johnson High School,77.2%,76.7%,77.5%,76.9%
Pena High School,83.6%,83.4%,84.3%,84.1%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [91]:
# Create a pandas series for each grade. Hint: use a conditional statement.
ninth = school_data_complete.loc[school_data_complete["grade"] == "9th"]
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th"]
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th"]
twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th"]

# Group each series by school
ninth_reading = ninth.groupby("school_name")["reading_score"].sum() / \
    ninth.groupby("school_name")["Student ID"].count()
tenth_reading = tenth.groupby("school_name")["reading_score"].sum() / \
    tenth.groupby("school_name")["Student ID"].count()
eleventh_reading = eleventh.groupby("school_name")["reading_score"].sum() / \
    eleventh.groupby("school_name")["Student ID"].count()
twelfth_reading = twelfth.groupby("school_name")["reading_score"].sum() / \
    twelfth.groupby("school_name")["Student ID"].count()

# Combine the series into a dataframe
reading_summary = pd.DataFrame ({ "School Name": "", "9th": [], "10th": [],"11th": [], "12th": []
})
reading_summary["School Name"] = school_data["school_name"]
reading_summary["9th"] = [x for x in ninth_reading]
reading_summary["10th"] = [x for x in tenth_reading]
reading_summary["11th"] = [x for x in eleventh_reading]
reading_summary["12th"] = [x for x in twelfth_reading]
reading_summary = reading_summary.set_index("School Name")
reading_summary.index.name = None

# Optional: give the displayed data cleaner formatting

reading_summary["9th"] = reading_summary["9th"].map("{:.1f}%".format)
reading_summary["10th"] = reading_summary["10th"].map("{:.1f}%".format)
reading_summary["11th"] = reading_summary["11th"].map("{:.1f}%".format)
reading_summary["12th"] = reading_summary["12th"].map("{:.1f}%".format)


reading_summary

,9th,10th,11th,12th
Bailey High School,81.3%,80.9%,80.9%,80.9%
Cabrera High School,83.7%,84.3%,83.8%,84.3%
Figueroa High School,81.2%,81.4%,80.6%,81.4%
Ford High School,80.6%,81.3%,80.4%,80.7%
Griffin High School,83.4%,83.7%,84.3%,84.0%
Hernandez High School,80.9%,80.7%,81.4%,80.9%
Holden High School,83.7%,83.3%,83.8%,84.7%
Huang High School,81.3%,81.5%,81.4%,80.3%
Johnson High School,81.3%,80.8%,80.6%,81.2%
Pena High School,83.8%,83.6%,84.3%,84.6%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [164]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [167]:
spending_summary = pd.DataFrame ({
    "Spending Ranges (Per Student)": "",
    "Average Math Score": [],
    "Average Reading Score": [],
    "% Passing Math": [],
    "% Passing Reading": [],
    "% Overall Passing Rate": []
})

spending_summary["Average Math Score"] = school_summary["Average Math Score"]
spending_summary["Average Reading Score"] = school_summary["Average Reading Score"]
spending_summary["% Passing Math"] = school_summary["% Passing Math"]
spending_summary["% Passing Reading"] = school_summary["% Passing Reading"]
spending_summary["% Overall Passing Rate"] = school_summary["% Overall Passing Rate"]
spending_summary["Spending Ranges (Per Student)"] = \
    pd.cut([i / j for i, j in zip(school_data["budget"], school_data["size"])], spending_bins, labels=group_names)
spending_summary = spending_summary.groupby("Spending Ranges (Per Student)")
spending_summary.max()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.803279,83.814988,92.505855,96.252927,94.379391
$585-615,83.351499,83.975780,94.133477,97.138965,95.586652
$615-645,83.682222,83.955000,93.867121,96.611111,94.972222
$645-675,83.839917,84.044699,94.594595,97.308869,95.290520


## Scores by School Size

* Perform the same operations as above, based on school size.

In [169]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [170]:
size_summary = pd.DataFrame ({
    "School Size": [],
    "Average Math Score": [],
    "Average Reading Score": [],
    "% Passing Math": [],
    "% Passing Reading": [],
    "% Overall Passing Rate": []
})

size_summary["Average Math Score"] = school_summary["Average Math Score"]
size_summary["Average Reading Score"] = school_summary["Average Reading Score"]
size_summary["% Passing Math"] = school_summary["% Passing Math"]
size_summary["% Passing Reading"] = school_summary["% Passing Reading"]
size_summary["% Overall Passing Rate"] = school_summary["% Overall Passing Rate"]
size_summary["School Size"] = pd.cut([i for i in school_summary["Total Students"]], size_bins, labels=group_names)
size_summary = size_summary.groupby("School Size")

size_summary.max()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.839917,84.044699,94.594595,96.252927,95.270270
Medium (1000-2000),83.682222,83.975780,94.133477,97.308869,95.586652
Large (2000-5000),83.274201,83.989488,93.867718,96.539641,95.203679


## Scores by School Type

* Perform the same operations as above, based on school type.